In [1]:
import cv2
import numpy as np
import pandas as pd
import os
from keras.optimizers import SGD, Adam
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.inception_v3 import InceptionV3
from keras.layers import Dense, Dropout, GlobalAveragePooling2D
from keras.models import Model
from keras.utils import np_utils
import keras.backend as K
from keras.callbacks import TensorBoard
from time import time

Using TensorFlow backend.


In [15]:
def make_generator(dataset):
    dataset_path = 'Data/pixabay/cats'

    datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                 rotation_range=40,
                                 width_shift_range=0.2,
                                 height_shift_range=0.2,
                                 zoom_range=0.2,
                                 horizontal_flip=True)
    return datagen.flow_from_directory(str(dataset_path),
                                       target_size=(299, 299),
                                       batch_size=32,
                                       class_mode='categorical')

In [2]:
base_model = InceptionV3(weights='imagenet', include_top=False)
base_output = base_model.output

avg_pool = Dropout(0.5)(GlobalAveragePooling2D(name='avg_pool')(base_output))
final_output = Dense(2, activation='softmax')(avg_pool)
model = Model(inputs=base_model.input, outputs=final_output)

for layer in base_model.layers:
    layer.trainable = False

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [3]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['binary_accuracy'])

In [16]:
tensorboard = TensorBoard(log_dir="logs/{}".format(time()))
train_generator = make_generator('train')
validation_generator = make_generator('test')
model.fit_generator(train_generator, steps_per_epoch=320, validation_data=validation_generator,
    validation_steps=64, epochs=10, verbose=1, callbacks=[tensorboard])

Found 41272 images belonging to 2 classes.
Found 41272 images belonging to 2 classes.
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
320/320 [==============================] - 943s 3s/step - loss: 0.5929 - binary_accuracy: 0.7054 - val_loss: 0.5947 - val_binary_accuracy: 0.6733
Epoch 2/10
320/320 [==============================] - 1009s 3s/step - loss: 0.5957 - binary_accuracy: 0.7012 - val_loss: 0.5518 - val_binary_accuracy: 0.7163
Epoch 3/10
320/320 [==============================] - 770s 2s/step - loss: 0.6032 - binary_accuracy: 0.6940 - val_loss: 0.5637 - val_binary_accuracy: 0.6982
Epoch 4/10
320/320 [==============================] - 1028s 3s/step - loss: 0.6007 - binary_accuracy: 0.6930 - val_loss: 0.5884 - val_binary_accuracy: 0.6729
Epoch 5/10
320/320 [==============================] - 992s 3s/step - loss: 0.5943 - binary_accuracy: 0.6967 - val_loss: 0.6438 - val_binary_accuracy: 0.6060
Epoch 6/10
 19/320 [>.............................] - ETA: 6:16 - loss: 0.5956 

C:\Users\pokem\Anaconda3\lib\site-packages\keras\callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.158998). Check your callbacks.
  % delta_t_median)


 20/320 [>.............................] - ETA: 6:41 - loss: 0.5981 - binary_accuracy: 0.6906

C:\Users\pokem\Anaconda3\lib\site-packages\keras\callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.160995). Check your callbacks.
  % delta_t_median)


 21/320 [>.............................] - ETA: 6:54 - loss: 0.5947 - binary_accuracy: 0.6935

C:\Users\pokem\Anaconda3\lib\site-packages\keras\callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.171998). Check your callbacks.
  % delta_t_median)


 23/320 [=>............................] - ETA: 7:41 - loss: 0.5902 - binary_accuracy: 0.7011

C:\Users\pokem\Anaconda3\lib\site-packages\keras\callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.152995). Check your callbacks.
  % delta_t_median)


320/320 [==============================] - 797s 2s/step - loss: 0.5959 - binary_accuracy: 0.6988 - val_loss: 0.5378 - val_binary_accuracy: 0.7373
Epoch 7/10
320/320 [==============================] - 899s 3s/step - loss: 0.5962 - binary_accuracy: 0.7018 - val_loss: 0.5588 - val_binary_accuracy: 0.7178
Epoch 8/10
320/320 [==============================] - 478s 1s/step - loss: 0.6020 - binary_accuracy: 0.6963 - val_loss: 0.5724 - val_binary_accuracy: 0.6885
Epoch 9/10
320/320 [==============================] - 482s 2s/step - loss: 0.5892 - binary_accuracy: 0.7016 - val_loss: 0.5601 - val_binary_accuracy: 0.7241
Epoch 10/10
320/320 [==============================] - 479s 1s/step - loss: 0.6030 - binary_accuracy: 0.6937 - val_loss: 0.5437 - val_binary_accuracy: 0.7246


In [4]:
# model.save_weights("cuteness.h5")
model.load_weights("cuteness.h5")

In [5]:
def image_resize(im, width, height):
    desired_size = width
    old_size = im.shape[:2] # old_size is in (height, width) format
    ratio = float(desired_size)/max(old_size)
    new_size = tuple([int(x*ratio) for x in old_size])

    # new_size should be in (width, height) format

    im = cv2.resize(im, (new_size[1], new_size[0]))

    delta_w = desired_size - new_size[1]
    delta_h = desired_size - new_size[0]
    top, bottom = delta_h//2, delta_h-(delta_h//2)
    left, right = delta_w//2, delta_w-(delta_w//2)

    color = [0, 0, 0]
    new_im = cv2.copyMakeBorder(im, top, bottom, left, right, cv2.BORDER_CONSTANT,
        value=color)
    
    return new_im

In [6]:
from matplotlib import pyplot as plt

# plt.imshow(im)
# plt.title('my picture')
# plt.show()
predictions = []
processed = 0
filecount = len(os.listdir("Data/train_images"))
for filename in os.listdir("Data/train_images"):
    im = image_resize(cv2.imread("Data/train_images/" + filename), 299, 299)
    im = im.astype(np.float32)
    im = np.expand_dims(im, axis=0)
    im = preprocess_input(im)
    
    processed = processed + 1
    scores = model.predict(im)
    predictions.append((filename[:9], scores[0][1]))
    if (processed % 100 == 0):
        print("Processed %d of %d" % (processed,filecount))

Processed 100 of 58311
Processed 200 of 58311
Processed 300 of 58311
Processed 400 of 58311
Processed 500 of 58311
Processed 600 of 58311
Processed 700 of 58311
Processed 800 of 58311
Processed 900 of 58311
Processed 1000 of 58311
Processed 1100 of 58311
Processed 1200 of 58311
Processed 1300 of 58311
Processed 1400 of 58311
Processed 1500 of 58311
Processed 1600 of 58311
Processed 1700 of 58311
Processed 1800 of 58311
Processed 1900 of 58311
Processed 2000 of 58311
Processed 2100 of 58311
Processed 2200 of 58311
Processed 2300 of 58311
Processed 2400 of 58311
Processed 2500 of 58311
Processed 2600 of 58311
Processed 2700 of 58311
Processed 2800 of 58311
Processed 2900 of 58311
Processed 3000 of 58311
Processed 3100 of 58311
Processed 3200 of 58311
Processed 3300 of 58311
Processed 3400 of 58311
Processed 3500 of 58311
Processed 3600 of 58311
Processed 3700 of 58311
Processed 3800 of 58311
Processed 3900 of 58311
Processed 4000 of 58311
Processed 4100 of 58311
Processed 4200 of 58311
P

Processed 33400 of 58311
Processed 33500 of 58311
Processed 33600 of 58311
Processed 33700 of 58311
Processed 33800 of 58311
Processed 33900 of 58311
Processed 34000 of 58311
Processed 34100 of 58311
Processed 34200 of 58311
Processed 34300 of 58311
Processed 34400 of 58311
Processed 34500 of 58311
Processed 34600 of 58311
Processed 34700 of 58311
Processed 34800 of 58311
Processed 34900 of 58311
Processed 35000 of 58311
Processed 35100 of 58311
Processed 35200 of 58311
Processed 35300 of 58311
Processed 35400 of 58311
Processed 35500 of 58311
Processed 35600 of 58311
Processed 35700 of 58311
Processed 35800 of 58311
Processed 35900 of 58311
Processed 36000 of 58311
Processed 36100 of 58311
Processed 36200 of 58311
Processed 36300 of 58311
Processed 36400 of 58311
Processed 36500 of 58311
Processed 36600 of 58311
Processed 36700 of 58311
Processed 36800 of 58311
Processed 36900 of 58311
Processed 37000 of 58311
Processed 37100 of 58311
Processed 37200 of 58311
Processed 37300 of 58311


In [7]:
pd_arr = np.array(predictions)

In [13]:
a = pd_arr
sidx = np.lexsort(a[:,[1,0]].T)
idx = np.append(np.flatnonzero(a[1:,0] > a[:-1,0]), a.shape[0]-1)    
out = a[sidx[idx]]
len(out)
np.savetxt('np_max.csv', out, fmt='%s', delimiter=',', header=" id, cute_val")